In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
import psycopg2
import random
import sys

In [2]:
def loadData(url):
    try:
        dataset = pd.read_csv(url,delimiter='\t',)
        print(dataset.shape)
        dict_uf= {11:'RO',12:'AC',13:'AM',14:'RR',15:'PA',16:'AP',17:'TO',21:'MA',22:'PI',23:'CE',24:'RN',
              25:'PB',26:'PE',27:'AL',28:'SE',29:'BA',31:'MG',32:'ES',33:'RJ',35:'SP',41:'PR',42:'SC',43:'RS',50:'MS',
              51:'MT',52:'GO',53:'DF',}
        dataset = dataset.replace({"ID_UF": dict_uf})
        return dataset;
    except:
        print("Oops!", sys.exc_info()[0], "occurred.")

In [3]:
path_file = '../Dataset/inep_saeb_merge_fatorial_2019.csv'
df = loadData(path_file)


(62668, 197)


In [4]:
dict_uf= {11:'RO',12:'AC',13:'AM',14:'RR',15:'PA',16:'AP',17:'TO',21:'MA',22:'PI',23:'CE',24:'RN',
              25:'PB',26:'PE',27:'AL',28:'SE',29:'BA',31:'MG',32:'ES',33:'RJ',35:'SP',41:'PR',42:'SC',43:'RS',50:'MS',
              51:'MT',52:'GO',53:'DF',}
df = df.replace({"ID_UF": dict_uf})
df.head()

,ID_SAEB,ID_REGIAO,ID_UF,CO_ENTIDADE,NU_PRESENTES_5EF,NU_PRESENTES_9EF,NU_PRESENTES_EMT,NU_PRESENTES_EMI,NU_PRESENTES_EM,MEDIA_5EF_LP,...,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,Estrutura,PED,LibrasBraile,TratamentoLixo,PCD,Acessibilidade,Internet,Transporte
0,2019,1,RO,11024666,18.0,13.0,NaN,NaN,NaN,184.83,...,0.0,0.0,-1.09,-0.29,-0.28,-0.22,0.29,-1.51,0.14,0.32
1,2019,1,RO,11024682,149.0,NaN,NaN,NaN,NaN,223.45,...,0.0,0.0,-0.97,-0.28,-0.50,0.73,1.53,-1.18,0.58,0.16
2,2019,1,RO,11024828,21.0,14.0,NaN,NaN,NaN,158.06,...,0.0,0.0,-1.06,-0.29,-0.14,-0.23,-0.50,-1.40,0.04,0.28
3,2019,1,RO,11024968,NaN,NaN,66.0,NaN,66.0,NaN,...,0.0,0.0,-1.01,-0.29,-0.17,-0.24,-0.70,-1.49,0.55,-0.17
4,2019,1,RO,11025077,13.0,9.0,NaN,NaN,NaN,173.45,...,0.0,0.0,-1.09,-0.30,-0.29,-0.22,0.08,-1.36,-0.03,0.33


In [5]:
#Agrupar por Nacional
#Veririfcar Estado com maior/menor pontuacao 
cols = ['Estrutura','PED','LibrasBraile','TratamentoLixo','PCD','Acessibilidade','Internet','Transporte']
#data_brasil_fatores = df[cols].mean()

data_brasil_fatores = pd.DataFrame(df[cols].mean().to_dict(),index=[df[cols].index.values[-1]])

data_brasil_fatores.head()

,Estrutura,PED,LibrasBraile,TratamentoLixo,PCD,Acessibilidade,Internet,Transporte
62667,-0.000006,-0.000005,0.000009,0.000014,0.000002,-0.000007,0.000005,-0.000011


In [6]:
#Agrupar por Estados
filter = (df['ID_UF'] == 'PA')
data_estado_ = df[filter]

cols = ['Estrutura','PED','LibrasBraile','TratamentoLixo','PCD','Acessibilidade','Internet','Transporte']


data_estado_1_fatores = pd.DataFrame(data_estado_[cols].mean().to_dict(),index=[data_estado_[cols].index.values[-1]])
data_estado_1_fatores.head()



,Estrutura,PED,LibrasBraile,TratamentoLixo,PCD,Acessibilidade,Internet,Transporte
5541,-0.575539,0.043245,0.30105,-0.024661,0.27091,-1.60798,-0.187623,-0.328272


In [7]:
#Agrupar por Estados
filter = (df['ID_UF'] == 'RS')
data_estado_ = df[filter]

cols = ['Estrutura','PED','LibrasBraile','TratamentoLixo','PCD','Acessibilidade','Internet','Transporte']


data_estado_2_fatores = pd.DataFrame(data_estado_[cols].mean().to_dict(),index=[data_estado_[cols].index.values[-1]])
data_estado_2_fatores.head()



,Estrutura,PED,LibrasBraile,TratamentoLixo,PCD,Acessibilidade,Internet,Transporte
58462,-0.57584,0.315529,-0.068447,0.03462,0.075431,1.146198,0.478045,-0.045082


In [22]:
def plotRadarCensoSaeb():
   
    #Carrega dados
    #path_file = 'https://raw.githubusercontent.com/eltonsarmanho/GoogleFitDataFlow/main/data/inep_saeb_merge_fatorial_2019.csv'
    path_file = '../Dataset/inep_saeb_merge_fatorial_2019.csv'

    dataset = pd.read_csv(path_file, delimiter='\t', )
    dict_uf = {11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO', 21: 'MA', 22: 'PI', 23: 'CE',
               24: 'RN',
               25: 'PB', 26: 'PE', 27: 'AL', 28: 'SE', 29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR',
               42: 'SC', 43: 'RS', 50: 'MS',
               51: 'MT', 52: 'GO', 53: 'DF', }
    df = dataset.replace({"ID_UF": dict_uf})
    
    #Processamento no contexto nacional
    #Agrupar por Nacional
    cols = ['Estrutura','PED','LibrasBraile','TratamentoLixo','PCD','Acessibilidade','Internet','Transporte']
    
    data_brasil_fatores = pd.DataFrame(df[cols].mean().to_dict(),index=[df[cols].index.values[-1]])

    
    dimensoes = list(data_brasil_fatores.columns)
    dimensoes = [*dimensoes, dimensoes[0]]

    esc = data_brasil_fatores.values.flatten().tolist()
    esc = [*esc, esc[0]]
    
    regiao_norte = ['AM','PA','AC','RR','RO','AP','TO']
    
    estado_1 = 'PA'
    #Processamento no contexto Estadual
    filter = (df['ID_UF'].isin(regiao_norte))
    data_estado_ = df[filter]

    data_estado_1_fatores = pd.DataFrame(data_estado_[cols].mean().to_dict(),index=[data_estado_[cols].index.values[-1]])

    
    dimensoes_estado_1 = list(data_estado_1_fatores.columns)
    dimensoes_estado_1 = [*dimensoes_estado_1, dimensoes_estado_1[0]]

    esc_estado_1 = data_estado_1_fatores.values.flatten().tolist()
    esc_estado_1 = [*esc_estado_1, esc_estado_1[0]]    
    
    
    #Processamento no contexto Escolar    
    regiao_sul = ['PR','SC','RS']
    
    estado_2 = 'MA'   
    
    filter = (df['ID_UF'].isin(regiao_sul))
    data_estado_ = df[filter]

    data_estado_2_fatores = pd.DataFrame(data_estado_[cols].mean().to_dict(),index=[data_estado_[cols].index.values[-1]])

    
    dimensoes_estado_2 = list(data_estado_2_fatores.columns)
    dimensoes_estado_2 = [*dimensoes_estado_2, dimensoes_estado_2[0]]

    esc_estado_2 = data_estado_2_fatores.values.flatten().tolist()
    esc_estado_2 = [*esc_estado_2, esc_estado_2[0]]    
    
    
    fig = go.Figure(
        data=[
            go.Scatterpolar(r=esc_estado_1, theta=dimensoes_estado_1, name='Região Norte'),
            go.Scatterpolar(r=esc_estado_2, theta=dimensoes_estado_2, name='Região Sul'),
            go.Scatterpolar(r=esc, theta=dimensoes, name='Nacional')
        
        ],
        layout=go.Layout(
            title_y=0.98,title_x=0.5,
            title=go.layout.Title(text='Fatores SAEB-CENSO'),
            polar={'radialaxis': {'visible': False}},
            showlegend=True
        )
    )
    return fig

In [23]:
fig = plotRadarCensoSaeb()
fig.show()